In [ ]:
#build using LSTM
#LSTM are used for sequential data
#sequential data = order of data matters
#use this tutorial https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html
'''
1. convert language to numeric label 
2. get an tokenizer for programming languages
3. tokenize all of my code
4. create pipelines
5. Dataloader
6. define model 
7. initiate instance
8. define functions to train the model and evaluate results
9. split dataset and run the model
10. Evaluate the model with test dataset
11. 

stuck
Problem:
    habe dataset and have to convert one column to tokens, the creation of this tokens should make sense print() as token = p,r,i,nt,() does not make sense but print, ()
    given is my training dataset with originally 3 columns
    my code colum is the column which needs to be tokenized
    the code is in a structure of several words
    i need to find out how i can tokenize the code based on that I can train a model which identifies pattersn like how which combinations of which tokens identify a certain label
    i also have to encode my target "language" to some numerical value
    i need an tokenizer who is specific for programmign languages
    I can get for every cell a list of the tokenized code, is that what i want? check in standard code if this has the same structure there
    
Plan:
    1. structure code (comments)
    2. fix bug of "too many values to unpack (expected 2)" so that I can transform my vocab to numbers to proceed
    
Problem:
    need to convert code column to tokens
Plan:
    1. iterate over code column andassign every cell new token value
        Problem:
            cannot manipulate dataframe with item = token
        Plan:
            (do everything with the test dataset "test" of 100 rows)
            0. structure code
            1. Google: How to create a copy of a dataframe and change his values
            2. get index of current row and than access item with info about column and row and convert item to token
            3. replacement..
                Problem:
                    i wanna replace a list of size 1 with an list > 1 which is not possible. can i replacer
                    
                Plan:
                    -1 do i later merge the list? checkout in. huggingface how the merge works
                    0. google Replacement lists must match in length. Expecting 1 got 72 --> how to replace lists with different lengths in pandas dataframe?
                    0.5 create a copy of a dataframe where every cell in the code column holds the encoded tokens
                    1. concatenate list at every comma with an Ä to list of size one will fail because sometime i need to replace the list again
                    2. wait for an answer of stackoverflow https://stackoverflow.com/questions/71697352/how-to-replace-a-value-of-smaller-length-than-one-with-bigger-length-in-pandas-d
                    3. check if in the standard modell the column which gets tokenized has also lists with such lengths
Plan:
    1. convert the whole column of the whole dataset to tokens
    2. 
     
'''

In [101]:
#import all libraries
import pandas as pd
import numpy as np
import os
from torchtext.data.utils import get_tokenizer
from transformers import RobertaTokenizer
from torchtext.vocab import build_vocab_from_iterator

In [102]:
#load and read datasets
trainData = pd.read_csv("data/train.csv")
testData = pd.read_csv("data/test.csv")

#use only code and language of dataset and (drop Id column)
trainData.head()

,id,code,language
0,14026,var result = testObj1 | testObj2;\...,c-sharp
1,12201,/// Initializes a new instance of ...,c-sharp
2,17074,/*\n\n Explanation :- a user gives a Strin...,javascript
3,21102,int sum = 0;\n\n for (int i = ...,c-plus-plus
4,53065,if (p->data < min)\n\n {\n\n ...,c


In [103]:
#get id from tokenizer which i use
CODEBERTA_LANGUAGE_ID = "huggingface/CodeBERTa-language-id"

In [104]:
#define tokenizer
tokenizer = RobertaTokenizer.from_pretrained(CODEBERTA_LANGUAGE_ID)

In [111]:
trainData["code"]=trainData["code"].apply(lambda x: tokenizer.encode(x))

In [113]:
print(trainData)

          id                                               code     language
0      14026  [0, 278, 722, 758, 272, 2651, 2642, 21, 631, 2...      c-sharp
1      12201  [0, 267, 23137, 262, 40719, 349, 426, 925, 465...      c-sharp
2      17074  [0, 3718, 203, 203, 262, 1647, 15327, 31411, 3...   javascript
3      21102  [0, 267, 664, 3090, 272, 430, 31, 203, 203, 26...  c-plus-plus
4      53065  [0, 267, 317, 298, 84, 303, 544, 596, 1591, 13...            c
...      ...                                                ...          ...
45623  77158  [0, 267, 1104, 272, 1144, 203, 203, 262, 858, ...       python
45624  19972  [0, 278, 407, 18, 352, 70, 67, 487, 18, 812, 1...       python
45625  45356  [0, 263, 853, 284, 203, 203, 264, 3143, 478, 3...  c-plus-plus
45626  20558  [0, 2824, 7963, 87, 31, 203, 203, 1946, 3671, ...         java
45627  31300  [0, 267, 5202, 376, 24637, 415, 203, 203, 264,...         java

[45628 rows x 3 columns]
